In [1]:
import os
import pickle

In [2]:

os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= '2'


In [17]:
from natsort import natsorted
from tqdm import tqdm
from glob import glob
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D

all_blocks = []
all_labels = []

all_files_path = natsorted(glob('merged_data_pkl/*'))

for path in tqdm(all_files_path):
    with open(path, 'rb') as file:
        data_dict = pickle.load(file)
        all_blocks.append(data_dict['data'])
        all_labels.append(data_dict['label'])

X = np.array(all_blocks)
Y = np.array(all_labels)

print(X.shape, Y.shape)


100%|██████████| 8400/8400 [00:00<00:00, 13483.90it/s]


(8400, 32, 250) (8400,)


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 10)


In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D
from sklearn.model_selection import train_test_split

In [20]:
from natsort import natsorted
from glob import glob
X_data_files  = natsorted(glob('./train_pkl/train_data_person_*_data.pkl'))
Y_label_files = natsorted(glob('./train_pkl/train_data_person_*_label.pkl'))

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms

class SoundCNN(nn.Module):
    def __init__(self):
        super(SoundCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(3, 3), padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(3, 3), padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 4 * 31, 512)  # Corrected dimension
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)  # Assuming 10 different classes
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [22]:
# Reshape data to include channel dimension (1, 32, 250)
X = X.reshape(X.shape[0], 1, 32, 250)

Y = Y.astype(np.int64)


# Convert to torch tensors
tensor_X = torch.Tensor(X)  # transform to torch tensor
tensor_Y = torch.Tensor(Y)

# Create your dataloader
my_dataset = TensorDataset(tensor_X, tensor_Y)
train_loader = DataLoader(my_dataset, batch_size=64, shuffle=True)


In [31]:
import torch

# Define the device globally
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def validation_accuracy(model, val_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total  # Correct percentage calculation

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, save_path, device):
    model.to(device)  # Ensure model is on the right device
    best_accuracy = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        val_acc = validation_accuracy(model, val_loader, device)
        print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}, Validation Accuracy: {val_acc}%')
        
        if val_acc > best_accuracy:
            best_accuracy = val_acc
            torch.save(model.state_dict(), save_path)  # Save only state dict for efficiency
            print(f'Saved better model at epoch {epoch + 1} with validation accuracy: {val_acc}%.')

# Example usage
for i in range(10):
    model = SoundCNN()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    save_path = 'path_to_save_model.pth'

    train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, save_path=save_path, device=device)


NameError: name 'SomeModelDefinition' is not defined

In [14]:
import os
import numpy as np
import pandas as pd
import pickle
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D

# Assuming X_data_files and Y_label_files are defined somewhere above this block
index = 0
for X_data_path, Y_label_path in zip(X_data_files, Y_label_files):
    filename = os.path.splitext(os.path.basename(X_data_path))[0].split('_')[-2]
    
    with open(X_data_path, 'rb') as file:
        X_data = pickle.load(file)
    with open(Y_label_path, 'rb') as file:
        Y_label = pickle.load(file)

    all_blocks = []
    all_labels = [] 

    skip_window_size = 32
    label_cnt = 0

    for idx in range(0, X_data.shape[0], skip_window_size):
        data_frame = X_data[idx:idx+skip_window_size, :]
        
        skip_frame_size = 50

        for frame_idx in range(0, data_frame.shape[1], skip_frame_size):
            if frame_idx != 0:
                data_chunk = data_frame[:, max(0, frame_idx-25):frame_idx+skip_frame_size]
            else:
                data_chunk = data_frame[:, frame_idx:frame_idx+skip_frame_size+25]
            
            all_blocks.append(data_chunk)
            all_labels.append(Y_label[label_cnt])
        
        label_cnt += 1
        
    X = np.array(all_blocks)
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)  # Reshaping for Conv2D input
    y = np.array(all_labels)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the CNN model
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(np.unique(y).size, activation='softmax')
    ])

    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train model
    model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

    # Save the trained model
    model.save(f'ensemble_models/trained_cnn_model_{index}.h5')
    index += 1


Epoch 1/10


2024-04-14 02:09:17.862882: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


 1/95 ━━━━━━━━━━━━━━━━━━━━ 3:35 2s/step - accuracy: 0.1250 - loss: 4.2142

I0000 00:00:1713040758.824874 1214382 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


95/95 ━━━━━━━━━━━━━━━━━━━━ 14s 126ms/step - accuracy: 0.1548 - loss: 3.5135 - val_accuracy: 0.2381 - val_loss: 2.0071
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - accuracy: 0.2866 - loss: 1.8770 - val_accuracy: 0.2751 - val_loss: 1.8637
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - accuracy: 0.3879 - loss: 1.6945 - val_accuracy: 0.3254 - val_loss: 1.8476
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 112ms/step - accuracy: 0.4802 - loss: 1.4680 - val_accuracy: 0.3386 - val_loss: 1.7433
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 111ms/step - accuracy: 0.5609 - loss: 1.2451 - val_accuracy: 0.3571 - val_loss: 1.7598
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 114ms/step - accuracy: 0.6806 - loss: 0.9723 - val_accuracy: 0.3862 - val_loss: 1.9332
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - accuracy: 0.7473 - loss: 0.7421 - val_accuracy: 0.3743 - val_loss: 2.1354
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - accuracy: 0.8701 - loss: 0.4595 - val_accuracy: 0.390

...

Epoch 1/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 14s 127ms/step - accuracy: 0.1194 - loss: 3.3430 - val_accuracy: 0.1561 - val_loss: 2.1834
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 116ms/step - accuracy: 0.1734 - loss: 2.1617 - val_accuracy: 0.2011 - val_loss: 2.1215
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 116ms/step - accuracy: 0.2614 - loss: 2.0329 - val_accuracy: 0.2513 - val_loss: 2.0480
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 115ms/step - accuracy: 0.3453 - loss: 1.8388 - val_accuracy: 0.2619 - val_loss: 2.0436
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 109ms/step - accuracy: 0.4231 - loss: 1.6298 - val_accuracy: 0.2698 - val_loss: 2.0569
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 108ms/step - accuracy: 0.5069 - loss: 1.4179 - val_accuracy: 0.2672 - val_loss: 2.2165
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - accuracy: 0.6174 - loss: 1.1521 - val_accuracy: 0.2685 - val_loss: 2.3013
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - accuracy: 0.7311 - loss: 0.8321 - val_accu

Epoch 1/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 13s 122ms/step - accuracy: 0.1193 - loss: 3.9385 - val_accuracy: 0.1892 - val_loss: 2.1364
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - accuracy: 0.2280 - loss: 2.0697 - val_accuracy: 0.2288 - val_loss: 2.0140
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - accuracy: 0.2870 - loss: 1.8707 - val_accuracy: 0.2659 - val_loss: 1.8884
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 112ms/step - accuracy: 0.4075 - loss: 1.6054 - val_accuracy: 0.3320 - val_loss: 1.6588
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 112ms/step - accuracy: 0.5297 - loss: 1.2443 - val_accuracy: 0.3823 - val_loss: 1.5420
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 111ms/step - accuracy: 0.6564 - loss: 0.9478 - val_accuracy: 0.4484 - val_loss: 1.4778
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 112ms/step - accuracy: 0.7431 - loss: 0.7190 - val_accuracy: 0.4206 - val_loss: 1.5694
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 111ms/step - accuracy: 0.8362 - loss: 0.5128 - val_accu

Epoch 1/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 13s 121ms/step - accuracy: 0.1295 - loss: 3.0971 - val_accuracy: 0.1389 - val_loss: 2.1938
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 106ms/step - accuracy: 0.1708 - loss: 2.1657 - val_accuracy: 0.1640 - val_loss: 2.1535
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 107ms/step - accuracy: 0.2028 - loss: 2.1037 - val_accuracy: 0.1759 - val_loss: 2.0934
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - accuracy: 0.3082 - loss: 1.9163 - val_accuracy: 0.2632 - val_loss: 1.9070
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - accuracy: 0.4121 - loss: 1.6339 - val_accuracy: 0.3016 - val_loss: 1.8827
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 110ms/step - accuracy: 0.5300 - loss: 1.3145 - val_accuracy: 0.3360 - val_loss: 1.8200
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - accuracy: 0.6671 - loss: 0.9723 - val_accuracy: 0.3386 - val_loss: 1.9592
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 112ms/step - accuracy: 0.7915 - loss: 0.6744 - val_accu

Epoch 1/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 14s 124ms/step - accuracy: 0.1591 - loss: 3.3907 - val_accuracy: 0.2937 - val_loss: 1.8837
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 116ms/step - accuracy: 0.3311 - loss: 1.7765 - val_accuracy: 0.3995 - val_loss: 1.6736
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 117ms/step - accuracy: 0.4673 - loss: 1.4693 - val_accuracy: 0.4458 - val_loss: 1.5353
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - accuracy: 0.5352 - loss: 1.2435 - val_accuracy: 0.4392 - val_loss: 1.5911
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - accuracy: 0.6512 - loss: 1.0053 - val_accuracy: 0.4206 - val_loss: 1.6635
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - accuracy: 0.7437 - loss: 0.7523 - val_accuracy: 0.4431 - val_loss: 1.7576
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 112ms/step - accuracy: 0.8287 - loss: 0.5341 - val_accuracy: 0.4696 - val_loss: 1.8719
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 114ms/step - accuracy: 0.8982 - loss: 0.3280 - val_accu

Epoch 1/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 14s 128ms/step - accuracy: 0.1442 - loss: 3.6580 - val_accuracy: 0.1733 - val_loss: 2.1464
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 111ms/step - accuracy: 0.2306 - loss: 2.0642 - val_accuracy: 0.3042 - val_loss: 1.8183
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 113ms/step - accuracy: 0.4187 - loss: 1.6425 - val_accuracy: 0.3505 - val_loss: 1.7749
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 112ms/step - accuracy: 0.4892 - loss: 1.4524 - val_accuracy: 0.3942 - val_loss: 1.6218
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 111ms/step - accuracy: 0.5606 - loss: 1.2171 - val_accuracy: 0.3929 - val_loss: 1.7112
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 11s 112ms/step - accuracy: 0.6469 - loss: 0.9956 - val_accuracy: 0.4193 - val_loss: 1.7298
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 109ms/step - accuracy: 0.7103 - loss: 0.8246 - val_accuracy: 0.3981 - val_loss: 1.8317
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - accuracy: 0.8299 - loss: 0.5631 - val_accu